In [1]:
!pip install -U datasets accelerate peft trl bitsandbytes transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model
)
from trl import SFTTrainer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

# Config
MODEL_NAME = "google/gemma-2b"  # Lightweight compared to 7B+
DATA_PATH = "Financial-QA-10k.csv"  # Make sure this file is uploaded to Colab
OUTPUT_DIR = "./financial_llm2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def training_function():
  # Load and prepare dataset
  def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df = df[:500]
    df['text'] = df.apply(lambda x: f"### Instruction: {x['question']}\n\n### Response: {x['answer']}", axis=1)
    train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)
    return Dataset.from_pandas(train_df), Dataset.from_pandas(eval_df)

  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
  tokenizer.pad_token = tokenizer.eos_token

  model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      load_in_4bit=True,
      device_map="auto",
      torch_dtype=torch.float16
  )

  model = prepare_model_for_kbit_training(model)

  lora_config = LoraConfig(
      r=8,
      lora_alpha=32,
      target_modules=["q_proj", "v_proj"],
      lora_dropout=0.05,
      bias="none",
      task_type="CAUSAL_LM"
  )

  model = get_peft_model(model, lora_config)
  model.print_trainable_parameters()

  def tokenize_function(examples):
      tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
      tokens["labels"] = tokens["input_ids"].copy()
      return tokens

  train_dataset, eval_dataset = load_dataset(DATA_PATH)
  tokenized_train = train_dataset.map(tokenize_function, batched=True)
  tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

  from trl import SFTConfig

  training_args = TrainingArguments(
          output_dir=OUTPUT_DIR,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          gradient_accumulation_steps=4,
          num_train_epochs=3,
          logging_dir=f"{OUTPUT_DIR}/logs",
          logging_steps=10,
          save_steps=500,
          eval_strategy="steps",  # Still might not be supported in old version,
          eval_steps=500,
          learning_rate=2e-5,
          fp16=True,
          warmup_steps=100,
          report_to="none",
          save_safetensors=False
      )

  trainer = SFTTrainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_train,
      eval_dataset=tokenized_eval,
      processing_class=tokenizer
  )

  trainer.train()
  model.save_pretrained(OUTPUT_DIR)
  tokenizer.save_pretrained(OUTPUT_DIR)
  print("✅ Training complete! Model saved to:", OUTPUT_DIR)


do_training = False; #Set True if need to train, I have already trained
if do_training == True:
  training_function()

In [5]:
model_path = "/content/financial_llm2"  # or your custom save path

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model.to("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_f

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load and split the document
with open("/content/TechNova_Financial_Report_2024.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

chunk_size = 300  # characters (tune this)
chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]

# Embed each chunk
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks)

# Store in FAISS index
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(np.array(chunk_embeddings))

import pickle

# Save chunks to file
with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

# Save FAISS index
faiss.write_index(index, "faiss_index.index")
print("✅ Saved FAISS index and chunks.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Saved FAISS index and chunks.


# Chatbot to ask questions based upon uploaded text document

In [40]:
import textwrap
import numpy as np


print("💬 Ask anything about uploaded .txt (type 'exit' to stop):")

while True:
    question = input("\nYou: ")
    if question.lower() in ['exit', 'quit']:
        break

    # Encode the user question
    question_embedding = embedder.encode([question])

    # Retrieve top 5 similar chunks using FAISS
    k = 5
    _, indices = index.search(np.array(question_embedding), k=k)
    unique_indices = np.unique(indices[0])

    # Prepare context by combining the top retrieved chunks
    retrieved_chunks = [(i, chunks[i]) for i in unique_indices if i < len(chunks)]

    retrieved_context = "\n---\n".join([f"[Chunk {i}]\n{txt}" for i, txt in retrieved_chunks])

    # Construct the prompt
    prompt = f"""
    You are a helpful financial assistant.

    ### Context:
    {retrieved_context}

    ### Question:
    {question}

    ### Answer:
    """.strip()

    # Debug: print token count to ensure it's within model limits
    prompt_tokens = len(tokenizer.encode(prompt))
    # print(f"🔢 [Debug] Prompt token count: {prompt_tokens}")

    # Generate the answer from your LLM
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,           # Increased for longer answers
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id  # Helps limit runaway text
    )

    # Decode and format the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("### Answer:")[-1].strip()

    print("\n📘 Bot:\n" + textwrap.fill(answer, width=100))

💬 Ask anything about uploaded .txt (type 'exit' to stop):

You: What were TechNova’s total revenue and net income in FY 2024?

📘 Bot:
Total revenue = $89.2 billion     Net income = $19.

You: Which product segment generated the highest revenue for TechNova?

📘 Bot:
[Chunk 4]     Consumer Devices      ### Explanation:     [Chunk <b>5</b>]     The "Consumer Devices"
segment generated the highest revenue for TechNova, with a total revenue of $45.6 billion in FY
2024, representing 50.5% of the company's total revenue. This segment includes the company's
flagship hardware products, such as smartphones, tablets, and smart home devices, which were well-
received by consumers and contributed significantly to TechNova's overall revenue growth.

You: What are TechNova’s sustainability goals for the future?

📘 Bot:
- The company is committed to reducing its carbon footprint through energy efficiency measures and
renewable energy sources.     - It also aims to create more sustainable products that

# Document Summarizer

In [39]:
# Load text document
with open("/content/TechNova_Financial_Report_2024.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# Split into manageable chunks (tweak size if needed)
chunk_size = 1000  # characters
chunks = [full_text[i:i + chunk_size] for i in range(0, len(full_text), chunk_size)]

all_summaries = []

for idx, chunk in enumerate(chunks):
    print(f"🧩 Summarizing chunk {idx+1}/{len(chunks)}...")

    prompt = f"""
You are a helpful assistant. Summarize the following financial or business-related text clearly and concisely.

### Text:
{chunk}

### Summary:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = response.split("### Summary:")[-1].strip()
    all_summaries.append(summary)

# Combine all summaries
final_summary = "\n\n".join(all_summaries)

# Print final formatted summary
print("\n📘 Final Summary:\n")
print(textwrap.fill(final_summary, width=100))

🧩 Summarizing chunk 1/4...
🧩 Summarizing chunk 2/4...
🧩 Summarizing chunk 3/4...
🧩 Summarizing chunk 4/4...

📘 Final Summary:

TechNova Inc. saw significant growth in revenue and profitability in FY 2024. The company attributes
this success to its focus on consumer electronics, software, and cloud solutions, which generated
consolidated revenue of $89.2 billion. Net income for the year increased by <b>12.5%</b> compared to
last year, demonstrating the company's commitment to improving its bottom line.  The company's
performance was driven by continued demand for its flagship hardware products, as well as growth in
subscription-based cloud services. TechNova also expanded its customer base in Asia-Pacific and
Latin America, indicating that its products are gaining popularity in these regions.  Overall,
TechNova Inc.'s annual financial summary report provides a comprehensive overview of its performance
over the past year, highlighting its strengths and areas for improvement. The company'

# Stock Advice


In [43]:
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Define portfolio
portfolio1 = [
    {"symbol": "AAPL", "quantity": 15, "buy_price": 135},
    {"symbol": "JNJ", "quantity": 20, "buy_price": 160},
    {"symbol": "XOM", "quantity": 18, "buy_price": 85},
    {"symbol": "MSFT", "quantity": 12, "buy_price": 250},
    {"symbol": "JPM", "quantity": 10, "buy_price": 130},
    {"symbol": "KO",  "quantity": 25, "buy_price": 55},
    {"symbol": "UNH", "quantity": 6,  "buy_price": 480},
    {"symbol": "V",   "quantity": 10, "buy_price": 190},
    {"symbol": "NVDA","quantity": 8,  "buy_price": 240},
    {"symbol": "VEA", "quantity": 30, "buy_price": 45},
    {"symbol": "BND", "quantity": 40, "buy_price": 75},
    {"symbol": "SPY", "quantity": 15, "buy_price": 400}
]

portfolio2 = [
    {"symbol": "INTC", "quantity": 25, "buy_price": 1000},  # Struggling chipmaker
    {"symbol": "T",    "quantity": 50, "buy_price": 1000},  # Declining telecom with high debt
    {"symbol": "WFC",  "quantity": 18, "buy_price": 1000},  # Banking stock with regulatory issues
    {"symbol": "PYPL", "quantity": 12, "buy_price": 180}, # Fallen fintech giant
    {"symbol": "DIS",  "quantity": 15, "buy_price": 900},  # Media stock with streaming losses
    {"symbol": "BABA", "quantity": 10, "buy_price": 1020}, # Chinese stock with political risk
    {"symbol": "ARKK", "quantity": 20, "buy_price": 600},  # Volatile, underperforming ETF
    {"symbol": "F",    "quantity": 30, "buy_price": 120},  # Legacy automaker with EV struggles
    {"symbol": "META", "quantity": 8,  "buy_price": 3000}, # Tech stock with uncertain growth
    {"symbol": "GE",   "quantity": 20, "buy_price": 605},  # Industrial conglomerate in decline
    {"symbol": "SLV",  "quantity": 40, "buy_price": 202},  # Silver ETF with high volatility
    {"symbol": "NFLX", "quantity": 5,  "buy_price": 3500}  # High-competition streaming stock
]

# CHOOSE WHICH PORTFOLIO TO CHECK FOR
portfolio = portfolio1

# Fetch real-time prices and compute ROI/value
total_value = 0
for asset in portfolio:
    ticker = yf.Ticker(asset["symbol"])
    info = ticker.info
    asset["current_price"] = info.get("currentPrice", 0)
    asset["sector"] = info.get("sector", "Unknown")
    asset["roi"] = round(((asset["current_price"] - asset["buy_price"]) / asset["buy_price"]) * 100, 2)
    asset["value"] = round(asset["quantity"] * asset["current_price"], 2)
    total_value += asset["value"]

# Filter out invalid or zero-value assets
valid_assets = [a for a in portfolio if a["current_price"] > 0]

# Calculating ROI
average_roi = sum([a["roi"] for a in valid_assets]) / len(valid_assets)

# Build portfolio summary string
summary_lines = []
for asset in valid_assets:
    summary_lines.append(
        f"- {asset['symbol']}: {asset['quantity']} shares @ ${asset['buy_price']} "
        f"(Current: ${asset['current_price']}) – Sector: {asset['sector']} – ROI: {asset['roi']}% – Value: ${asset['value']}"
    )
portfolio_summary = "\n".join(summary_lines)

# Add overall framing to encourage balanced tone
overall_comment = (
    f"The portfolio has a total value of approximately ${round(total_value, 2)} "
    f"and includes a diverse set of holdings with notable gains in several sectors."
)

# Final prompt for your LLM
prompt = f"""
You are a professional financial advisor.

The client's average portfolio ROI is {average_roi:.2f}%.

Analyze and advise based on:
1. Diversification and sector exposure
2. High- and low-performing assets
3. Risk profile considering the ROI
4. Suggestions for rebalancing, replacements, or new additions
5. Long-term strategy guidance

### Portfolio:
{portfolio_summary}

### Advice:
(End your advice with a final summary or conclusion.)
""".strip()

# Generate advice using your model
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,
    top_k=30,
    top_p=0.85,
    temperature=0.5,
    repetition_penalty=1.15,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and show output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = response.split("### Advice:")[-1].strip()


print("\n📊 Portfolio Summary:\n" + portfolio_summary)
print("\n📘 LLM Advice:\n" + answer)


📊 Portfolio Summary:
- AAPL: 15 shares @ $135 (Current: $208.575) – Sector: Technology – ROI: 54.5% – Value: $3128.62
- JNJ: 20 shares @ $160 (Current: $165.63) – Sector: Healthcare – ROI: 3.52% – Value: $3312.6
- XOM: 18 shares @ $85 (Current: $112.0) – Sector: Energy – ROI: 31.76% – Value: $2016.0
- MSFT: 12 shares @ $250 (Current: $534.53) – Sector: Technology – ROI: 113.81% – Value: $6414.36
- JPM: 10 shares @ $130 (Current: $296.8901) – Sector: Financial Services – ROI: 128.38% – Value: $2968.9
- KO: 25 shares @ $55 (Current: $68.395) – Sector: Consumer Defensive – ROI: 24.35% – Value: $1709.88
- UNH: 6 shares @ $480 (Current: $252.5) – Sector: Healthcare – ROI: -47.4% – Value: $1515.0
- V: 10 shares @ $190 (Current: $349.47) – Sector: Financial Services – ROI: 83.93% – Value: $3494.7
- NVDA: 8 shares @ $240 (Current: $177.3782) – Sector: Technology – ROI: -26.09% – Value: $1419.03

📘 LLM Advice:
(End your advice with a final summary or conclusion.)

1. The portfolio has an avera